## Initialization of environemnt, imports

In [1]:
#from __future__ import absolute_import
#from __future__ import print_function
#from __future__ import division

from IPython.display import display, HTML

from importlib import reload

import pandas as pd
import json, os, re, shutil, sys, time
import multiprocessing as mp
from multiprocessing import Queue
from multiprocessing import Pool
import collections, itertools
import unittest
import datetime
import string
import pickle
import string
import copy

from cytoolz import concatv

# NLTK for NLP utils and corpora
import nltk,pprint
from nltk import word_tokenize
nltk.download('treebank')
nltk.download('punkt')

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

#parameter setting
work_dir = "working_dir/"

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/i812749/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /Users/i812749/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def sent_tokenize_rm_punct (blob):
    #input is string
    #remove punctuations
    #change all to lower case
    #change 0-9 to N (we keep the number of digits so 321 will be NNN this way system will try to generate NNN in the approximate magnitute)
    #output is a list of strings (using ,?!, as senence spliter and split the sentence)
    blst = []   
    bexp=''
    blob.replace('\n','')
    #blob.replace('</p>','  ')

    for i, char in (enumerate(blob)):
        #print(i,char)
        next_cap = False
        prev_lower = False
        next_space = False
        
        if char in string.punctuation :
            #print("found punctuation:", i, char, blob[i+1])
           
            if char in '.?!' :
                if i+1 >= len(blob):
                    next_cap = False
                else:
                    next_cap = blob[i+1].isupper()
                    next_space = blob[i+1].isspace()    


                if i-1 <0:
                    prev_lower = False
                else:
                    prev_lower = blob[i-1].islower()

                if (next_cap and prev_lower) or next_space : 
                    # if the char before "." is lower case, 
                    #but the one immediately follow the "." is Uppercase, 
                    #then this is a paragraph end (caused by removing <p></p> from the xml file)
                    #of course if ". " period followed by space is end of sentence.
                    ##if bexp != '' :
                    ##    blst = blst + [bexp]
                    ##bexp=''
                    
                    #mark "period" here as this is likely real end of sentence.
                    bexp=bexp + ' <p> '
                else:
                    #if this is not end of paragraph, then 
                    #bexp = bexp + char
                    pass
            else :
                bexp = bexp + ' '
        elif char.isspace() :
            if i-1 >=0 :
                if blob[i-1].isspace() :
                    pass
                else :
                    bexp = bexp + char.lower()
        elif char == '\n':
            pass

        else :
            if char.isnumeric() : 
                bexp = bexp + 'N'
            else :
                bexp = bexp + char.lower()
    if bexp != '' : 
        blst = blst + [bexp]
        
    
    return(blst)




In [ ]:
class NYTData:
    __author__ = "Alan Tan"
    __copyright__ = "Copyright 2018, Alan Tan"
    __credits__ = ["Alan Tan"]
    __license__ = "Apache"
    __version__ = "2.0"
    __updatedby__ = "Alan Tan, 2018"
    
    def __init__(self, ):
        self.source_raw=[]
        self.target_raw=[]
        self.source_token_ids=[]
        self.target_token_ids=[]
        self.vocab_to_int={'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 , '<p>': 4}
        self.int_to_vocab={0 : '<PAD>', 1: '<EOS>', 2: '<UNK>', 3: '<GO>', 4 : '<p>'}

        
    def load_from_file(self, input_file_path='nyt_dataset/abstract_nyt_structured_data.csv'):
        #read file into Pandas Dataframe.
        #each line is one record, it contains "original file name(including y/m/d)","TITLE","ABSTRACT","LEAD_PARAGRAPH","FULL_TEXT"
        nyt_data = pd.read_csv(input_file_path, 
                               delimiter=',', index_col=0, header=None, quotechar='"', quoting=1, 
                               skipinitialspace=True, engine='c')


        #We will use LEAD_PARAGRAPH as source for now
        #And ABSTRACT as target. 
        # Remove punctuation, change digits to NNN, and to lower case.     
        self.source_raw=nyt_data[3]
        self.target_raw=nyt_data[2]
        print("Read",len(self.source_raw),"records into source text.")
        print("Read",len(self.target_raw),"records into target text.")
        
    def preprocess(self, max_sentence_length=np.inf):
        # make a list of unique words
        CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 , '<p>': 4}

        self.source_token_ids = []
        self.target_token_ids = []
        
        pool = Pool()
        source_text = pool.map(sent_tokenize_rm_punct, self.source_raw)
        # serial mode, no parallel multiprocessing .# source_text= sum([sent_tokenize_rm_punct(sentlst) for sentlst in self.source_raw],[])
        print("source_text length" , len(source_text))
        target_text = pool.map(sent_tokenize_rm_punct, self.target_raw)
        
        #target_text= sum([sent_tokenize_rm_punct(sentlst) for sentlst in self.target_raw],[])
        print("target_text length", len(target_text)) 
        pool.close()
        pool.join()
        
        source_short_text,target_short_text=zip(*([(st, tt) for st, tt, in zip(source_text, target_text) if len(st.split())<=max_sentence_length*2 and len(tt.split())<=max_sentence_length]))
        source_short_length = [len(tt.split()) for tt in source_short_text]
        target_short_length = [len(tt.split()) for tt in target_short_text]
        print("only", len(source_short_text), "source sentences are less than",max(source_short_length), "long.")
        print("only", len(target_short_text), "target sentences are less than",max(target_short_length), "long.")
        
        
        #This is a summary model, so we create *one* vocab, (unlike translate model, where 2 separate vocab would be needed)
        vocab = set(' '.join(source_text).split()).union(' '.join(target_text).split())
        print("vocab size", len(vocab))

        # initially vocab starts with the special tokens, and we build all words into the dict vocab_to_int
        self.vocab_to_int = copy.copy(CODES)
        for w_i, w in enumerate(vocab, len(CODES)):
            self.vocab_to_int[w] = w_i
            
        # create reverse lookup dict(from index to word)
        self.int_to_vocab = {w_i: w for w, w_i in self.vocab_to_int.items()}

        print("vocab dict built:",len(self.vocab_to_int))
        
        # create token_ids which are intger indexes of words in our vocab (after tokenization and lower case)
        for source_sentence in source_short_text :
            source_token_id = [self.vocab_to_int[s] for s in source_sentence.split()]
            self.source_token_ids.append(source_token_id)
        print("source_token_ids size:",len(self.source_token_ids))
            
        for target_sentence in target_short_text :
            target_token_id = [self.vocab_to_int[s] for s in target_sentence.split()]
            self.target_token_ids.append(target_token_id)        
        print("target_token_ids size:",len(self.target_token_ids))
        

    def infer_prep(self,infer_sentences, pad=True):
        sentences_int=[]
        max_sentence_length=0
        for sentence in sum([sent_tokenize_rm_punct(sent) for sent in infer_sentences],[]) :
            
            sentence_int=[]
            for word in sentence.split() :
                if word in self.vocab_to_int :
                    sentence_int.append(self.vocab_to_int[word])
                else :
                    sentence_int.append(self.vocab_to_int['<UNK>'])
            sentences_int.append(sentence_int)
            max_sentence_length=max(max_sentence_length,len(sentence_int))
            
        if pad :

            """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
            output_ids = [sentence_int + [self.vocab_to_int['<PAD>']] * (max_sentence_length - len(sentence_int)) for sentence_int in sentences_int]
        else :
            output_ids = sentences_int
        return output_ids


    def get_batches(self, batch_size):
        """Batch targets, sources, and the lengths of their sentences together"""
        for batch_i in range(0, len(self.sources_token_ids)//batch_size):
            start_i = batch_i * batch_size

            # Slice the right amount for the batch
            sources_batch = self.sources_token_ids[start_i:start_i + batch_size]
            targets_batch = self.sources_token_ids[start_i:start_i + batch_size]

            # Pad
            max_sentence = max([len(sentence) for sentence in sources_batch])
            pad_sources_batch = np.array([sentence + [self.vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sources_batch])
            max_sentence = max([len(sentence) for sentence in targets_batch])
            pad_targets_batch = np.array([sentence + [self.vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in targets_batch])

            # Need the lengths for the _lengths parameters
            pad_targets_lengths = []
            for target in pad_targets_batch:
                pad_targets_lengths.append(len(target))

            pad_sources_lengths = []
            for source in pad_sources_batch:
                pad_sources_lengths.append(len(source))

            yield pad_sources_batch, pad_targets_batch, pad_sources_lengths, pad_targets_lengths


        
    def save_preprocess(self, save_file_name='preprocess.p'):    
        # Save data for later use
        pickle.dump(self, open(save_file_name, 'wb'))
        
        print("Saved (", len(self.source_token_ids),  len(self.target_token_ids) , ") source , target tokens")
        print("   ...and (",len(self.vocab_to_int),") voc-int dictionary")
        print("   ...and (",len(self.int_to_vocab),") int-voc dictionary")
    
    def load_preprocess(self, load_file_name='preprocess.p'):
        with open(load_file_name, mode='rb') as in_file:
            
            self=pickle.load(in_file)        
        
    

In [31]:
if __name__ == '__main__':
    NYTDATA=NYTData()
    NYTDATA.load_from_file('nyt_dataset/short_abstract_nyt_structured_data.csv')
    NYTDATA.preprocess(max_sentence_length=50)
    
    with open(work_dir+'short_preprocess.p', 'wb') as f:
        pickle.dump(NYTDATA, f)
    #to load back
    #with open('first_40k_preprocess.p', 'rb') as f:
    #    NYTDATA = pickle.load(f)

Read 580454 records into source text.
Read 580454 records into target text.
source_text length 580454
target_text length 580454
only 572440 source sentences are less than 200 long.
only 572440 target sentences are less than 100 long.
vocab size 332193
vocab dict built: 332197
source_token_ids size: 572440
target_token_ids size: 572440


In [4]:

#with open(work_dir+'short_preprocess.p', 'rb') as f:
#        NYTDATA = pickle.load(f)
#max([len(text) for text in NYTDATA.source_token_ids])

200

29014


## Routines to build Graph

In [5]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len


In [6]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [7]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [16]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """

    #decoder_initial_state = dec_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)
    #decoder_initial_state = encoder_state

    #print("decoder_init_state is type:", type(decoder_initial_state))
    #print(decoder_initial_state)
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              initial_state=encoder_state, 
                                              output_layer=output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [17]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    
    #decoder_initial_state = encoder_state
    #decoder_initial_state = dec_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)

    
    #dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
    #                                         output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              initial_state=encoder_state, 
                                              output_layer=output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [20]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size, enc_outputs):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    #print(dec_input)
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    #print(dec_embed_input)
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    dec_cell = tf.contrib.rnn.DropoutWrapper(cells, output_keep_prob=keep_prob)
    if use_attention :
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units=batch_size, memory=enc_outputs) 
                #,                memory_sequence_length=150)
        encoder_state = dec_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)
    
        dec_cell = tf.contrib.seq2seq.AttentionWrapper(
                dec_cell, attention_mechanism, attention_layer_size=150)
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [21]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, (enc_states) = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    #print(type(enc_states))
    #print(len(enc_states))
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                               num_layers,
                                               target_vocab_to_int,
                                               target_vocab_size,
                                               batch_size,
                                               keep_prob,
                                               dec_embedding_size, enc_outputs)
    
    return train_output, infer_output

## Train

In [22]:
#Subroutines used in training
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths
        
        
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))


#Subrountines used in preparing data for inference. 
def sentence_to_seq(sentence, vocab_to_int):
    results = []
    for word in sentence.split(" "):
        if word in vocab_to_int:
            results.append(vocab_to_int[word])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

In [23]:
display_step = 100

epochs = 10
batch_size = 50

rnn_size = 300
num_layers = 3

encoding_embedding_size = 300
decoding_embedding_size = 300

use_attention=False
learning_rate = 0.0025
keep_probability = 0.9
save_path = "working_dir/"


#NYTDATA.load_preprocess()

max_target_sentence_length = max([len(sentence) for sentence in NYTDATA.source_token_ids])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    
    #lr, keep_prob = hyperparam_inputs()
    lr = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
  

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(NYTDATA.vocab_to_int),
                                                   len(NYTDATA.vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   NYTDATA.vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph prepared...")

Graph prepared...


In [24]:
# Split data to training and validation sets
print("start training...", datetime.datetime.now())

train_source = NYTDATA.source_token_ids[batch_size:]
train_target = NYTDATA.target_token_ids[batch_size:]
valid_source = NYTDATA.source_token_ids[:batch_size]
valid_target = NYTDATA.target_token_ids[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             NYTDATA.vocab_to_int['<PAD>'],
                                                                                                             NYTDATA.vocab_to_int['<PAD>']))   

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print(".",end='')
    for epoch_i in range(epochs):
        print(".")
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            NYTDATA.vocab_to_int['<PAD>'],
                            NYTDATA.vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 0.95})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 0.95})

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)
                
                print(datetime.datetime.now())
                print('\nEpoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(NYTDATA.source_token_ids) // batch_size, train_acc, valid_acc, loss))
                print('  source: {}'.format([NYTDATA.int_to_vocab[i] for i in valid_sources_batch[int((1+epoch_i)*(batch_i/100))%batch_size]]))
                print('  target: {}'.format([NYTDATA.int_to_vocab[i] for i in batch_valid_logits[int((1+epoch_i)*(batch_i/100))%batch_size]]))

        # Save checkpoint of training
        saver = tf.train.Saver()
        saver.save(sess, save_path+'ckpt', global_step=epoch_i)
        print('Model training checkpoing for epoch_{} is saved'.format(epoch_i))                   
    
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')        

start training... 2018-12-07 09:48:45.816655


KeyboardInterrupt: 

## inference

In [137]:
    #test inference_logits
with open(work_dir+'100words_or_less_preprocess.p', 'rb') as f:
        NYTDATA2 = pickle.load(f)
    
test_sentences_raw = [' '.join(shortern.split()[0:100]) for shortern in NYTDATA2.target_raw[0:0+batch_size]]

test_sentences = NYTDATA2.infer_prep(test_sentences_raw, pad=True)[0:batch_size]

test_sentence_length = [len(ts) for ts in test_sentences]   


[94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94]


In [138]:
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(save_path + '.meta')
    loader.restore(sess, save_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    inference_logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    
    translate_logits = sess.run(inference_logits, {input_data: test_sentences,
                                         target_sequence_length: test_sentence_length,
                                         keep_prob: 0.95})
    for ii in range(len(test_sentences)) :
        print('\n{}'.format(test_sentences_raw[ii]))
        #print('  Word Ids:      {}'.format([i for i in test_sentences[ii]]))
        print('\n\nInput: {}'.format([NYTDATA2.int_to_vocab[i] for i in test_sentences[ii]]))

        #print('\nOutput')
        #print('  Word Ids:      {}'.format([i for i in translate_logits[ii]]))
        print('  \nOutput: {}'.format(" ".join([NYTDATA2.int_to_vocab[i] for i in translate_logits[ii]])))
    #finished test infer
    

INFO:tensorflow:Restoring parameters from working_dir/

New York City Dept of Health and Mental Hygiene's efforts to compile detailed portrait of city's health runs into unexpected trouble; canvassers report problems trying to get representative sample, especially in well-off neighborhoods where people are too busy, too wary and just too hard to reach; photos (M)


Input: ['new', 'york', 'city', 'dept', 'of', 'health', 'and', 'mental', 'hygiene', 's', 'efforts', 'to', 'compile', 'detailed', 'portrait', 'of', 'city', 's', 'health', 'runs', 'into', 'unexpected', 'trouble', '<UNK>', 'report', 'problems', 'trying', 'to', 'get', 'representative', 'sample', 'especially', 'in', 'well', 'off', 'neighborhoods', 'where', 'people', 'are', 'too', 'busy', 'too', 'wary', 'and', 'just', 'too', 'hard', 'to', 'reach', 'photos', 'm', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '

In [ ]:
print(test_text[0])

In [94]:
#print(test_sentences[:][0:100])
print([len(shorten[0:100]) for shorten in NYTDATA2.target_raw[0:0+batch_size]])

[100, 100, 100, 75, 71, 100, 100, 100, 100, 100, 100, 100, 95, 100, 73, 100, 100, 100, 100, 35, 86, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 65, 100, 100, 100, 100, 100, 100, 89, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 89, 100, 100, 100, 100, 100, 100, 100, 100, 64, 100, 74, 100, 100, 89, 100, 100, 100, 100, 100, 100, 55, 100, 100, 100, 100, 100, 100, 100, 100, 15]


In [81]:
print(len(NYTDATA2.vocab_to_int))

59743
